# ETHIC - Primavera 2024

##  Selección de caso

* 1) Caso Amanda

In [1]:
from utils.extract_data import process_data
from utils.extract_data import read_data

caso = process_data()
df1, df2 = read_data(caso)


Casos disponibles:
1. Amanda
Caso seleccionado: Amanda
Archivo: Procesamient_2023_CD1201_2.xlsx, Año: 2023, Sección: 2
Archivo: Procesamient_2023_CD1201_3.xlsx, Año: 2023, Sección: 3
Archivo: Procesamient_2023_CD1201_4.xlsx, Año: 2023, Sección: 4
Archivo: Procesamient_2023_CD1201_5.xlsx, Año: 2023, Sección: 5
Archivo: Procesamient_2023_CD1201_7.xlsx, Año: 2023, Sección: 7
Archivo: Procesamient_2023_CD1201_8.xlsx, Año: 2023, Sección: 8
Archivo: Procesamient_2023_CD1201_9.xlsx, Año: 2023, Sección: 9
Archivo: Procesamient_2024_CD1100_2.xlsx, Año: 2024, Sección: 2
Archivo: Procesamient_2024_CD1100_4.xlsx, Año: 2024, Sección: 4
Archivo: Procesamient_2024_CD1100_6.xlsx, Año: 2024, Sección: 6
Archivo: Procesamient_2024_CD1100_7.xlsx, Año: 2024, Sección: 7
Archivo: Procesamient_2024_CD1100_8.xlsx, Año: 2024, Sección: 8

Carpeta ya existe: processed_data/Amanda
Datos procesados guardados en: processed_data/Amanda/answers_by_secc_Amanda.csv

Datos leídos de: processed_data/Amanda/answers_by_secc

## Analisis Gramatical